In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>", fig.align = "center"
)

In [ ]:
# Install Google Colab dependencies
# Note: this can take 30+ minutes (many of the dependencies include C++ code, which needs to be compiled)

# First install `sf`, `ragg` and `textshaping` and their system dependencies:
system("apt-get -y update && apt-get install -y  libudunits2-dev libgdal-dev libgeos-dev libproj-dev libharfbuzz-dev libfribidi-dev")
install.packages("sf")
install.packages("textshaping")
install.packages("ragg")

# Install system dependencies of some other R packages that Voyager either imports or suggests:
system("apt-get install -y libfribidi-dev libcairo2-dev libmagick++-dev")

# Install Voyager from Bioconductor:
install.packages("BiocManager")
BiocManager::install(version = "3.16", ask = FALSE, update = FALSE, Ncpus = 2)
BiocManager::install("scater")
system.time(
  BiocManager::install("Voyager", dependencies = TRUE, Ncpus = 2, update = FALSE)
)

packageVersion("Voyager")

# Introduction

Nanostring GeoMX DSP is a popular spatial transcriptomics technology for formalin fixed paraffin embedded (FFPE) tissues, but it doesn't have single cell resolution. The CosMX FISH based technology for FFPE tissue [@He2021-oy] does have single cell resolution, and this vignette provides an example of how to analyze CosMX data with voyager. Note that FFPE is a common way to preserve and archive tissue, and in some cases, the only samples available may be FFPE. 

The CosMX dataset for non-small cell lung cancer that we used is described in [@He2021-oy]. The processed data is available for download from the [Nanostring website](https://nanostring.com/products/cosmx-spatial-molecular-imager/ffpe-dataset/). The gene count matrix, cell metadata, and cell segmentation polygon coordinates were downloaded from the Nanostring website as CSV files and read into R as data frames. The gene count matrix was then converted to a sparse matrix. The cell metadata contains centroid coordinates of the cells. The cell polygon data frames were converted into an `sf` data frame with the `df2sf()` function in `SpatialFeatureExperiment` (SFE). These were then used to construct an SFE object. Cell segmentation is only available in one z-plane.

In [ ]:
library(Voyager)
library(SFEData)
library(SingleCellExperiment)
library(SpatialExperiment)
library(scater) # devel version of plotExpression
library(scran)
library(bluster)
library(ggplot2)
library(patchwork)
library(stringr)
library(spdep)
library(BiocParallel)
library(BiocSingular)
theme_set(theme_void())

In [ ]:
(sfe <- HeNSCLCData())

Only the first biological replicate is included in the `SFEData` package. This biological replicate has 980 features and 100,290 cells. Take a look at the cells in space:

In [ ]:
ggplot(cellSeg(sfe)) + geom_sf() +
    theme_bw() +
    scale_x_continuous(expand = expansion()) +
    scale_y_continuous(expand = expansion())

With single cell resolution, a lot of the details can be seen, although there's some artifact from borders of fields of view (FOVs).

Plot cell density

In [ ]:
plotCellBin2D(sfe) + theme_bw()

# Quality control (QC)
## Cells

Single cell RNA-seq (scRNA-seq) technologies typically don't quantify cell morphology, and gene expression in Visium doesn't have single cell resolution. Here for single cell resolution smFISH based data, each cell not only has gene expression and related QC metrics such as total number of transcripts detected and number of genes detected, but also cell morphology such as area (in the z-plane where the segmentation polygons are provided) and aspect ratio. Area is relevant to QC since it can flag falsely undersegmented cells, i.e. several cells falsely considered as one by the cell segmentation program. However, since a pre-defined gene panel is used and mitochondrially encoded genes are not quantified, the scRNA-seq QC metric of proportion of mitochondrially encoded counts is not applicable. 

Some QC metrics are precomputed and are stored in `colData`

In [ ]:
names(colData(sfe))

Cell area, aspect ratio, marker and stain intensities, i.e. all columns before "sample_id" come from Nanostring's website. The `sf` package can compute areas of the cell polygons. In R, the `EBImage` package can compute more morphological metrics such as aspect ratio, eccentricity, orientation, and etc., but it requires the data to be converted to raster. OpenCV can compute more morphological metrics for polygons without converting to raster, but it needs to be called from Python or C++. Since the math behind many basic morphological metrics is pretty simple, we may add those to `Voyager` in a future version.

Since plotting 100,000 polygons is slow and the plot isn't large enough for us to see the polygons anyway, we use [`scattermore`](https://github.com/exaexa/scattermore) to rasterize the plot to speed up plotting. Instead of plotting every single point, now `ggplot` merely displays a rasterized image.

In [ ]:
# Function to plot violin plot for distribution and spatial at once
plot_violin_spatial <- function(sfe, feature) {
    violin <- plotColData(sfe, feature, point_fun = function(...) list())
    spatial <- plotSpatialFeature(sfe, feature, colGeometryName = "centroids",
                                  scattermore = TRUE)
    violin + spatial +
        plot_layout(widths = c(1, 2))
}

Number of transcript spots detected per cell

In [ ]:
plot_violin_spatial(sfe, "nCounts")

In [ ]:
summary(sfe$nCounts)

To make nCounts and nGenes more comparable across datasets, we divide them by the number of genes probed. In this dataset, there are 960 genes, and 20 negative controls. However, because different genes may be probed in different datasets, which can be from different tissues, this does not make nCounts and nGenes completely comparable across datasets. However, it may still be somewhat comparable, since genes highly expressed in major cell types in the tissue tend to be selected for the gene panel.

In [ ]:
n_panel <- 960
colData(sfe)$nCounts_normed <- sfe$nCounts/n_panel
colData(sfe)$nGenes_normed <- sfe$nGenes/n_panel

In [ ]:
plotColDataHistogram(sfe, c("nCounts_normed", "nGenes_normed")) +
    theme_bw()

This means the cells mostly have less than 1 transcript count per gene on average, which is not surprising since not all cells express all genes. Most cells are detected to express less than 30% of all genes probed.

Number of genes (out of 980) detected per cell

In [ ]:
plot_violin_spatial(sfe, "nGenes")

In [ ]:
summary(sfe$nGenes)

Based on the spatial plot, it seems that nCounts and nGenes are biologically relevant, but there are cells with no transcripts detected. 

How nCounts relates to nGenes

In [ ]:
plotColDataBin2D(sfe, "nCounts", "nGenes") +
    theme_bw()

What's the nature of the cells without transcripts?

In [ ]:
colData(sfe)$is_empty <- colData(sfe)$nCounts < 1

In [ ]:
plotSpatialFeature(sfe, "is_empty", "cellSeg")

The cells without transcripts are in the central cavity.

In [ ]:
plotColData(sfe, x = "Area", y = "is_empty")

The "empty" cells tend to be smaller than other cells but there are also some really large ones. 

Cell area distribution

In [ ]:
plot_violin_spatial(sfe, "Area")

Larger cells are more likely to be found in certain areas of the tissue. It could be biological, or that under-segmentation is more likely for that cell type or that tissue region.

How does area relate to total counts?

In [ ]:
plotColDataBin2D(sfe, "nCounts", "Area") + theme_bw()

While there may vaguely seem that cells with more total counts tend to be larger (at least in this z-plane), there are some cells that are large but have low total counts.

Negative control probes are used in this dataset for QC. Here we calculate the proportion of transcripts attributed to the negative controls.

In [ ]:
neg_inds <- str_detect(rownames(sfe), "^NegPrb")
# Number of negative control probes
sum(neg_inds)
colData(sfe)$prop_neg <- colSums(counts(sfe)[neg_inds,])/colData(sfe)$nCounts

In [ ]:
plot_violin_spatial(sfe, "prop_neg")

The NA's are empty cells, and the proportion is very low except for a few outliers. How does prop_neg relate to nCounts?

In [ ]:
plotColDataBin2D(sfe, "nCounts", "prop_neg") +
    scale_fill_viridis_c() + theme_bw()

This looks kind of like the proportion of mitochondrial counts vs. nCounts plot for scRNA-seq, where cells with fewer total counts tend to have higher proportion of mitochondrial counts.

In [ ]:
# The zeros are removed
plotColDataHistogram(sfe, "prop_neg") +
    scale_x_log10() +
    theme_bw()

The distribution is not obviously bimodal, and since the x-axis is log transformed to better visualize the distribution, the 0's have been removed. It's kind of arbitrary; for now we'll remove cells with more than 10% of transcripts from negative controls.

In [ ]:
# Remove low quality cells
(sfe <- sfe[,!sfe$is_empty & sfe$prop_neg < 0.1])

After removing the low quality cells, there are 100,095 cells left.

### Markers
Nanostring provides some cell stain and marker intensities in the cell metadata.

In [ ]:
names(colData(sfe))

Here we plot aspect ratio and mean intensity of cells stains and markers, which have not been plotted before. PanCK is a marker for epithelial cells. CD45 is a leukocyte marker. CD3 is a T cell marker. Since it takes quite a while to plot 100,000 cells 6 times, `scattermore` really helps.

In [ ]:
plotSpatialFeature(sfe, c("AspectRatio", "Mean.DAPI", "Mean.MembraneStain", 
                          "Mean.PanCK", "Mean.CD45", "Mean.CD3"),
                   colGeometryName = "centroids", ncol = 2, scattermore = TRUE)

## Genes

In [ ]:
rowData(sfe)$means <- rowMeans(counts(sfe))
rowData(sfe)$vars <- rowVars(counts(sfe))
rowData(sfe)$is_neg <- neg_inds

In [ ]:
plotRowDataBin2D(sfe, "means", "vars", subset = "is_neg", bins = 50) +
    geom_abline(slope = 1, intercept = 0, color = "red") +
    scale_x_log10() + scale_y_log10() +
    annotation_logticks() +
    coord_equal() +
    theme_bw()

The red line $y = x$ is expected from Poisson data. Gene expression in this dataset has more variance than expected from Poisson, even for gene with lower expression. Zoom into the negative controls

In [ ]:
as.data.frame(rowData(sfe)[neg_inds,]) |> 
    ggplot(aes(means, vars)) +
    geom_point() +
    geom_abline(slope = 1, intercept = 0, color = "red") +
    scale_x_log10() + scale_y_log10() +
    annotation_logticks() +
    coord_equal() +
    theme_bw()

Among the "high quality" cells, the negative controls still have higher variance relative to mean compared to Poisson. 

Negative controls vs. real genes

In [ ]:
plotRowData(sfe, x = "means", y = "is_neg") +
    scale_y_log10() +
    annotation_logticks(sides = "b")

The negative controls have lower mean "expression" than the vast majority of real genes.

# Spatial autocorrelation in QC metrics
A spatial neighborhood graph is required for spatial dependence analyses with `spdep`. Without a benchmark, we don't yet know which type of neighborhood graph is the best for which purpose. 

Methods to find spatial neighborhood graphs in `spdep` other than `knearneigh()` (k nearest neighbors), `dnearneigh()` (find cells within a certain distance), and `poly2nb()` (polygon contiguity) are not recommended for larger datasets. While cell-cell contact may be biologically relevant, because cell segmentation is imperfect, leading to non-contiguous cell segmentation polygons for cells that appear contiguous in H&E, only using `poly2nb()` to find polygon contiguity neighbors without supplementing with another kind of neighborhood is problematic. Delaunay triangulation with the `deldir` package, which is used by `spdep` (`tri2nb()`), takes 4 to 5 minutes for a dataset of this size, but the run time increases much more drastically than linearly as the number of cells increases. Sphere of Interest (SOI) graph (`soi.graph()`) prunes edges from triangulation that are too long, and does not take long itself. So triangulation and SOI graph, while slower than `knearneigh()`, `dnearneigh()`, and `poly2nb()`, are somewhat practical considerations. The implementation of `gabrielneigh()` and `relativeneigh()` take impracticably long (over an hour and I terminated the R session out of impatience) for this dataset so are not recommended.

Methods to find approximate nearest neighbors such as Annoy (`AnnoyParam()`) and HNSW (`HnswParam()`), as supported by the [`bluster`](https://bioconductor.org/packages/release/bioc/html/bluster.html) and [`BiocNeighbors`](https://bioconductor.org/packages/release/bioc/html/BiocNeighbors.html) packages might speed up finding these graphs, but we haven't formally benchmarked them. 

See [Chapter 14 of Spatial Data Science](https://r-spatial.org/book/14-Areal.html) on proximity in areal data for a more detailed discussion of the different neighborhood graphs in `spdep`. The methods for areal data are the first to be wrapped by Voyager because much of spatial transcriptomics data is analogous to areal geospatial data, where data from several cells are aggregated over areas, which happens in Visium spots. Just like in geospatial areal data, the Visium aggregation areas are arbitrary and do not represent the underlying spatial process. Although sometimes geographical areal units are not arbitrary, that tissues are generally not in hexagonal grids means that Visium spot polygons are arbitrary in this context. Regions of interest (ROI) selection spatial transcriptomics methods, such as laser capture microdissection (LCM) and GeoMX DSP are more obviously analogous to geospatial areal data. The aggregation also happens when we analyze smFISH-based data at the cell level, if the basic unit of observation is individual transcript spots. 

While `spdep` caters to areal data, `gstat` caters to geostatistical data, where a continuous spatial process is sampled at point locations. In some ways, spatial transcriptomics data is analogous to geostatistical data. Visium samples the supposed spatial biological process in a regular hexagonal grid, if we pretend that Visium spots are points. In smFISH-based single cell resolution data, the cells observed can be thought of as a sample of an underlying spatial biological process supervening on the specific locations of the cells. In a sense, the cells are not samples, since smFISH based technologies attempt to visualize all cells in the tissue section. However, as the biological function of the tissue does not depend on this particular spatial arrangement of individual cells (i.e. supervenes on this particular spatial arrangement), but more of cell types, the specific cell locations observed can be thought of as samples of the process, if we consider the cell the basic unit of the spatial process. 

In the next release of Voyager, we plan to add semivariograms (from the [`gstat`](https://cran.r-project.org/web/packages/gstat/index.html) package) as an exploratory tool to identify the presence of spatial autocorrelation, its length scale, and anisotropy (i.e. different in different directions). Covariates can be specified when computing the variogram to account for spatial trends and adjust for another spatial variable. However, unlike Morans's I, the semivariogram can't identify negative spatial autocorrelation, although since the spatial neighborhood graph typically does not encode spatial directions, `spdep` autocorrelation metrics can't identify anisotropy. Another problem with the semivariogram is that it assumes that the data is intrinsically stationary, i.e. the semivariogram holds in the entire dataset, or that similarity between two cells only depends on their distance from each other, which may not be the case when spatial autocorrelation varies in space as evident for some genes in local spatial analyses. 

Single cell smFISH based data is also dissimiliar to both areal and geostatistical data in important ways. In geospatial areal data, data from numerous basic units of the spatial process (e.g. people in epidemiology) are aggregated over areas (e.g. cities), whereas in histological space, the cell is arguably a more sensible basic unit of the biological spatial process than individual mRNA molecules. Unlike in geostatistical data, the cells as seen in the tissue section are often polygons tessellating the tissue section rather than points. Furthermore, while ideally the samples of the underlying spatial process should not affect the spatial process itself in geostatistical data, the cells play active roles in the biological spatial process. 

However, data analysis methods for areal and geostatistical data can still be relevant to EDA and descriptive models (not causal or mechanistic) of single cell smFISH data. Different types of spatial neighborhood graphs for cells may be relevant to different processes. For instance, contiguity of cell segmentation polygons is relevant when contact is involved in cell signaling, although cell segmentation is imperfect. Positive spatial autocorrelation here can arise from contact activation, and negative autocorrelation can arise from contact inhibition. However, cells may also be influenced by longer range factors such as secreted ligands, morphogens, and simpler spatial trends like distance to artery and vein. In this case, perhaps the semivariogram and using Euclidean distance between cells as spatial weights in spatial autocorrelation metrics would be more relevant for EDA. It would be interesting to compare the results from different spatial neighborhood graphs and spatial weights, and between `spdep` and `gstat`. Perhaps there is no one best method, but different methods reveal different phenomena. The problem of choosing a spatial neighborhood matrix has a long history far predating spatial transcriptomics. See [@Getis2009-iw] for a brief discussion of decades of work around this issue.

Spatial autocorrelation metrics seek to measure how nearby things tend to be more similar or dissimilar, and the neighborhood graph and edge weights define what we mean by "nearby" in areal data. Note that because each Visium spot can contain from several to dozens of cells, the spatial neighborhood graphs of Visium spots describe neighborhood relationships of much longer length scales than spatial neighborhood graphs of single cells, so spatial autocorrelation metrics using the Visium graph have different meanings from cellular neighborhood graphs. 

For now, just to demonstrate software usage, we use the k nearest neighborhood graph with distance based edge weights, as commonly done in graph based clustering in scRNA-seq, although we don't yet know the best value for k in each scenario. For the purpose of this vignette, say use $k = 5$, and the execution time isn't outrageous. The argument `style = "W"` is to row normalize the adjacency matrix of the spatial neighborhood graph as this is necessary for Moran scatter plot. Inverse distance edge weights can take small values but what matter is the relative rather than the absolute values as the distance itself is in arbitrary unit; row normalizing the adjacency matrix makes the weighted average value from the neighbors comparable to the value at the cell itself. In this tissue, many cells appear contiguous, but since cell segmentation is imperfect, there are many false singletons, which makes polygon contiguity neighbors from `poly2nb()` problematic without some modification. But based on the distribution of the number of neighbors based on contiguity, $k = 5$ doesn't seem to be a bad approximate for contiguity.

In [ ]:
system.time(
    colGraph(sfe, "knn5") <- findSpatialNeighbors(sfe, method = "knearneigh",
                                                  dist_type = "idw", k = 5, 
                                                  style = "W")
    )

Now compute Moran's I for some cell QC metrics

In [ ]:
features_use <- c("nCounts", "nGenes", "Area", "AspectRatio")
sfe <- colDataMoransI(sfe, features_use, colGraphName = "knn5")

In [ ]:
colFeatureData(sfe)[features_use,]

Positive spatial autocorrelation is suggested, which is stronger in nCounts and nGenes.

What are the length scales of spatial autocorrelation for these QC metrics? It would be nice if the lagged neighborhood graphs can be stored and reused for all features rather than recomputed for each feature as in `spdep::sp.correlogram()` called behind the scene here. This takes a few minutes to run, but not as long as a typical song. Another way to find the length scale of spatial autocorrelation is to bin the cells into bins of different sizes and then find spatial autocorrelation at each bin size, which probably is faster than finding lagged values at higher and higher neighborhoods since `geom_bin2d()` and `geom_hex()` in `ggplot2` run pretty fast even for large datasets. Or use a semivariogram; `gstat` also bins the data when estimating the semivariogram and calculating the semivariogram over very long distance is much faster than the correlogram with cell-cell neighborhood graphs. 

In [ ]:
system.time(
    sfe <- colDataUnivariate(sfe, "sp.correlogram", features = features_use,
                         colGraphName = "knn5", order = 6, zero.policy = TRUE,
                         BPPARAM = MulticoreParam(2))
)

Note that `MulticoreParam()` doesn't work on Windows; this vignette was built on Linux. Use `SnowParam()` or `DoparParam()` for Windows. See `?BiocParallelParam` for the available parallel processing backends. We did not notice significant performance differences between `ShowParam()` and `MulticoreParam()` in this context.

In [ ]:
plotCorrelogram(sfe, features_use) +
    theme_bw()

They seem to have similar length scales, but aspect ratios tend to decay more quickly.

Moran's scatter plot for nCounts. 

In [ ]:
sfe <- colDataUnivariate(sfe, "moran.plot", "nCounts", colGraphName = "knn5")

In the first panel, the density is for all points in this plot, and in the second, the points influential to fitting the line are highlighted in red, still a 2D histogram to avoid overplotting.

In [ ]:
p1 <- moranPlot(sfe, "nCounts", binned = TRUE, plot_influential = FALSE)
p2 <- moranPlot(sfe, "nCounts", binned = TRUE)
p1 / p2 + plot_layout(guides = "collect") & theme_bw()

There are no obvious clusters in this plot but the clusters may be obscured by overplotting.

Local Moran's I for nCounts

In [ ]:
sfe <- colDataUnivariate(sfe, "localmoran", "nCounts", colGraphName = "knn5")

In [ ]:
plotLocalResult(sfe, "localmoran", "nCounts", colGeometryName = "cellSeg",
                divergent = TRUE, diverge_center = 0)

Cool, it appears that the epithelial regions tend to be more homogenous in nCounts.

# Data normalization
Given that there may be some relationship between cell size and total counts, and that total counts may be biological and thus not purely treated as technical, questions are raised about data normalization and how it should be different from the standard scRNA-seq practices. For instance, what are technical contributions to total counts for this kind of data? Furthermore, what to do with cell area, since part of it is technical, in where the z-plane of cell segmentation polygons intersects each cell, but for some cell types, it could be biological? Also, how would different methods of data normalization affect spatial autocorrelation? Should spatial autocorrelation be used in some ways when normalizing data? Besides correcting for technical effects and making gene expression in cells with different total counts more comparable, data normalization stabilizes variance and tries to make the data more normally distributed since many statistical methods assume normally distributed data. So while we don't know the best practice to normalize this kind of data, we will still normalize the data for downstream analyses.

In [ ]:
sfe <- logNormCounts(sfe)

# Moran's I

Here we run global Moran's I on log normalized gene expression.

In [ ]:
# Note: on your computer, you can put progressbar = TRUE inside MulticoreParam()
# to show progress bar. This applies to any BiocParallParam.
sfe <- runMoransI(sfe, features = rownames(sfe), 
                  BPPARAM = MulticoreParam(2))

Do real genes tend to have more spatial autocorrelation than negative controls?

In [ ]:
plotRowData(sfe, x = "moran_sample01", y = "is_neg") +
    geom_hline(yintercept = 0, linetype = 2)

It seems that at least at the shorter length scale captured by the k nearest neighbor graph, most genes don't have strong spatial autocorrelation while some have strong positive spatial autocorrelation. In contrast, Moran's I for the negative controls is closely packed around 0, indicating lack of spatial autocorrelation, which is a good sign, that there is no evidence of a technical artifact that manifests as a spatial trend manifest in the negative controls.

What are the genes with the highest Moran's I?

In [ ]:
top_moran <- rownames(sfe)[order(rowData(sfe)$moran_sample01, decreasing = TRUE)[1:6]]
plotSpatialFeature(sfe, top_moran, colGeometryName = "centroids", 
                   scattermore = TRUE, ncol = 2)

They all highlight the same epithelial regions. It could be that other regions are not as spatially organized, or that a short length scale is used for Moran's I here but the correlogram shows that Moran's I decays after the first order neighbors. I wonder how using a longer length scale would change the results.

# Non-spatial dimension reduction and clustering

In [ ]:
set.seed(29)
sfe <- runPCA(sfe, ncomponents = 30, scale = TRUE, BSPARAM = IrlbaParam())

In [ ]:
ElbowPlot(sfe, ndims = 30) + theme_bw()

In [ ]:
plotDimLoadings(sfe, dims = 1:6) + theme_bw()

In [ ]:
spatialReducedDim(sfe, "PCA", 6, colGeometryName = "centroids", divergent = TRUE,
                  diverge_center = 0, ncol = 2, scattermore = TRUE)

The first PC highlights the epithelium. PC2 highlights T cells. PC4 might highlight other leukocytes. Need to check the genes with the highest loadings to find what the other PCs mean.

In [ ]:
set.seed(29)
sfe <- runUMAP(sfe, dimred = "PCA", n_dimred = 15)

Non-spatial clustering and locating the clusters in space

In [ ]:
colData(sfe)$cluster <- clusterRows(reducedDim(sfe, "PCA")[,1:15],
                                    BLUSPARAM = SNNGraphParam(
                                        cluster.fun = "leiden",
                                        cluster.args = list(
                                            resolution_parameter = 0.5,
                                            objective_function = "modularity")))

In [ ]:
plotPCA(sfe, ncomponents = 4, colour_by = "cluster")

In [ ]:
plotUMAP(sfe, colour_by = "cluster")

In [ ]:
plotSpatialFeature(sfe, "cluster", colGeometryName = "cellSeg")

Further analyses that can be done at this stage:

* Which and how many cell types are in the neighborhood of each cell? This is subject to the different definitions of the neighborhood.
* Which cell types tend to co-localize where?
* Find spatial regions based on cell type colocalization, which can be done with the R package [`spicyR`](https://bioconductor.org/packages/release/bioc/html/spicyR.html) [@Canete2022-tz]

# Differential expression
Cluster marker genes are found with Wilcoxon rank sum test as commonly done for scRNA-seq.

In [ ]:
markers <- findMarkers(sfe, groups = colData(sfe)$cluster,
                       test.type = "wilcox", pval.type = "all", direction = "up")

It's already sorted by p-values

In [ ]:
markers[[6]]

Get the the significant marker for each cluster to plot. Since there're too many points, here we used the development version of `scater` to not to plot the points, which are uninformative due to overplotting and make this plot really slow.

In [ ]:
genes_use <- vapply(markers, function(x) rownames(x)[1], FUN.VALUE = character(1))
plotExpression(sfe, genes_use, x = "cluster", point_fun = function(...) list())

Plot more top marker genes in a heatmap

In [ ]:
genes_use2 <- unique(unlist(lapply(markers, function(x) rownames(x)[1:5])))
plotGroupedHeatmap(sfe, genes_use2, group = "cluster", colour = scales::viridis_pal()(100))

# Local spatial statistics of marker genes
Plot those genes in space

In [ ]:
plotSpatialFeature(sfe, genes_use, colGeometryName = "centroids", ncol = 2,
                   scattermore = TRUE)

Moran's I of these marker genes

In [ ]:
rowData(sfe)[genes_use, "moran_sample01", drop = FALSE]

Local Moran's I of these marker genes

In [ ]:
sfe <- runUnivariate(sfe, "localmoran", features = genes_use, colGraphName = "knn5",
                     BPPARAM = MulticoreParam(2))

In [ ]:
plotLocalResult(sfe, type = "localmoran", features = genes_use, 
                colGeometryName = "centroids", ncol = 2, divergent = TRUE,
                diverge_center = 0, scattermore = TRUE)

It seems that some histological regions tend to be more spatially homogenous in gene expression than others. The epithelial region tends to be more homogenous.

Run local spatial heteroscdasticity (LOSH) for these marker genes to find local heterogeneity

In [ ]:
sfe <- runUnivariate(sfe, "LOSH", features = genes_use, colGraphName = "knn5",
                     BPPARAM = MulticoreParam(2))

In [ ]:
plotLocalResult(sfe, type = "LOSH", features = genes_use, 
                colGeometryName = "centroids", ncol = 2, scattermore = TRUE)

Some genes are more heterogeneous where they are also more highly expressed, such as COLA1 and IGKC. However this is not the case for all genes. For example, MZT2A is quite ubiqiutously experssed, but is more heterogeneous in some regions than others, and KRT19 does not seem to be much more heterogeneous where it's more highly expressed. For MZT2A, LOSH picked up the artifact of the edges of the FOVs, although this is not apparent for other genes plotted here. Here we don't have information on which cell belongs to which FOV, but FOV edge effects should be considered in data normalization. It would be interesting to more systematically see how LOSH relates to gene expression across more genes, and how this differs in cell types and gene functions.

# Session Info

In [ ]:
sessionInfo()

# References